In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
store_data = pd.read_csv("Data/WMdata_with_holidays copy.csv")
store_data.head()

2023-10-26 11:21:55.733875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Name
0,0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,NaN
1,1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,Valentine's Day
2,2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,NaN
3,3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,NaN
4,4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,NaN


In [2]:
# Drop the non-beneficial ID column
store_data.drop(columns = ['Unnamed: 0'], inplace=True)
store_data

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Name
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,NaN
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,Valentine's Day
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,NaN
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,NaN
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,NaN
...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684,NaN
6431,45,2012-10-05,733455.07,0,64.89,3.985,192.170412,8.667,NaN
6432,45,2012-10-12,734464.36,0,54.47,4.000,192.327265,8.667,NaN
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667,NaN


In [3]:
# Determine the number of unique values in each column.
for i in store_data.columns:
    print(i, len(store_data[i].unique()))

Store 45
Date 143
Weekly_Sales 6435
Holiday_Flag 2
Temperature 3528
Fuel_Price 892
CPI 2145
Unemployment 349
Holiday_Name 8


In [8]:
# Look at Temperature value counts for binning
temp_counts = store_data['Temperature'].value_counts()
temp_counts.head(30)

50.43    11
67.87    10
76.67     9
76.03     9
70.28     9
72.62     9
50.56     8
64.21     8
64.05     8
78.47     7
44.42     7
58.97     7
50.81     7
62.62     7
40.65     7
70.19     7
70.87     7
49.96     7
52.27     7
82.10     6
71.64     6
77.49     6
44.04     6
70.93     6
68.40     6
52.77     6
75.00     6
62.96     6
87.01     6
65.74     6
Name: Temperature, dtype: int64

In [20]:
# Choose a cutoff value and create a list of temperatures to be replaced
# use the variable name `temps_to_replace`
temps_to_replace = list(temp_counts[temp_counts<5].index)

# Replace in dataframe
for temp in temps_to_replace:
    store_data['Temperature'] = store_data['Temperature'].replace(temp,"Other")

# Check to make sure binning was successful
store_data['Temperature'].value_counts()

Other    5827
50.43      11
67.87      10
76.03       9
70.28       9
         ... 
75.93       5
43.43       5
59.15       5
61.24       5
81.52       5
Name: Temperature, Length: 107, dtype: int64

In [34]:
# Look at Fuel_Price value counts for binning
fuel_counts = store_data['Fuel_Price'].value_counts()
fuel_counts.tail(20)

2.716    21
3.501    21
3.617    21
3.533    21
3.571    21
3.57     21
2.805    20
3.286    20
2.732    20
2.983    20
3.561    20
3.877    20
2.77     20
3.372    20
2.826    20
2.725    20
3.684    20
3.743    20
2.735    20
3.554    20
Name: Fuel_Price, dtype: int64

In [35]:
# Choose a cutoff value and create a list of Fuel_Price rates to be replaced
# use the variable name `fuel_to_replace`
fuel_to_replace = list(fuel_counts[fuel_counts<23].index)

# Replace in dataframe
for fuel in fuel_to_replace:
    store_data['Fuel_Price'] = store_data['Fuel_Price'].replace(fuel,"Other")

# Check to make sure binning was successful
store_data['Fuel_Price'].value_counts()

Other    5950
3.638      39
3.63       34
2.771      29
3.891      29
2.72       28
3.524      28
3.594      28
3.666      27
3.523      27
3.129      25
2.78       25
3.842      25
3.112      24
3.899      24
3.065      24
3.236      23
3.452      23
3.62       23
Name: Fuel_Price, dtype: int64

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
store_dummies = pd.get_dummies(store_data)
store_dummies.head()

,Store,Weekly_Sales,Holiday_Flag,CPI,Date_2010-02-05,Date_2010-02-12,Date_2010-02-19,Date_2010-02-26,Date_2010-03-05,Date_2010-03-12,...,Fuel_Price_3.899,Fuel_Price_Other,Unemployment_Other,Holiday_Name_Christmas Day,Holiday_Name_Easter,Holiday_Name_Halloween,Holiday_Name_Independence Day,Holiday_Name_New Year's Day,Holiday_Name_Thanksgiving Day,Holiday_Name_Valentine's Day
0,1,1643690.90,0,211.096358,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,1,1641957.44,1,211.242170,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
2,1,1611968.17,0,211.289143,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,1,1409727.59,0,211.319643,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
4,1,1554806.68,0,211.350143,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0


In [37]:
# Split our preprocessed data into our features and target arrays
X = store_dummies.drop('Weekly_Sales', axis=1)
y = store_dummies['Weekly_Sales']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X.columns)
hidden_node1 = 80
hidden_node2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                22480     
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 25761 (100.63 KB)
Trainable params: 25761 (100.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
# Create a callback that saves the model's weights every five epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path= 'checkpoints/weights.{epoch}.hdf5'

# Create callback
callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [42]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[callback])

Epoch 1/50
151/151 [==============================] - 2s 3ms/step - loss: -43728236.0000 - accuracy: 0.0000e+00
Epoch 2/50
151/151 [==============================] - 1s 3ms/step - loss: -708577472.0000 - accuracy: 0.0000e+00
Epoch 3/50
151/151 [==============================] - 0s 3ms/step - loss: -3386902784.0000 - accuracy: 0.0000e+00
Epoch 4/50
151/151 [==============================] - 0s 3ms/step - loss: -9498836992.0000 - accuracy: 0.0000e+00
Epoch 5/50
146/151 [============================>.] - ETA: 0s - loss: -19970985984.0000 - accuracy: 0.0000e+00
Epoch 5: saving model to checkpoints/weights.5.hdf5
151/151 [==============================] - 1s 3ms/step - loss: -20205922304.0000 - accuracy: 0.0000e+00
Epoch 6/50
151/151 [==============================] - 0s 3ms/step - loss: -36552671232.0000 - accuracy: 0.0000e+00
Epoch 7/50
151/151 [==============================] - 0s 3ms/step - loss: -59413176320.0000 - accuracy: 0.0000e+00
Epoch 8/50
151/151 [==============================

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

51/51 - 0s - loss: -1.6653e+13 - accuracy: 0.0000e+00 - 400ms/epoch - 8ms/step
Loss: -16652655656960.0, Accuracy: 0.0
